In [4]:
import tensorflow as tf
import tensorflow.keras as keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
from tqdm import tqdm
from numpy.fft import fft, fftshift
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tqdm import tqdm
import optuna
from scipy.integrate import cumtrapz
import warnings
warnings.filterwarnings(action='ignore')

print(tf.__version__) 

2.5.0


#### Data loading

In [5]:
train=pd.read_csv('./train_features.csv')
train_labels=pd.read_csv('./train_labels.csv')
test=pd.read_csv('./test_features.csv')

submission=pd.read_csv('./sample_submission.csv')

pd.options.display.max_columns=50

In [6]:
train.shape

(1875000, 8)

#### Feature engneering

#####  가속도, 자이로, (자이로-가속도) 센서값을 에너지로 표현

In [7]:
train['acc_Energy']=(train['acc_x']**2+train['acc_y']**2+train['acc_z']**2)**(1/3)
test['acc_Energy']=(test['acc_x']**2+test['acc_y']**2+test['acc_z']**2)**(1/3)

train['gy_Energy']=(train['gy_x']**2+train['gy_y']**2+train['gy_z']**2)**(1/3)
test['gy_Energy']=(test['gy_x']**2+test['gy_y']**2+test['gy_z']**2)**(1/3)

train['gy_acc_Energy']=((train['gy_x']-train['acc_x'])**2+(train['gy_y']-train['acc_y'])**2+(train['gy_z']-train['acc_z'])**2)**(1/3)
test['gy_acc_Energy']=((test['gy_x']-test['acc_x'])**2+(test['gy_y']-test['acc_y'])**2+(test['gy_z']-test['acc_z'])**2)**(1/3)

In [8]:
train.shape

(1875000, 11)

###### id별 데이터는 0.02초마다 측정된 값들이기 때문에 이전 시간 대비 변화량 적용

In [9]:
dt=0.02 
def jerk_signal(signal): 
        return np.array([(signal[i+1]-signal[i])/dt for i in range(len(signal)-1)])

In [10]:
train_dt=[]
for i in tqdm(train['id'].unique()):
    temp=train.loc[train['id']==i]
    for v in train.columns[2:]:
        values=jerk_signal(temp[v].values)
        values=np.insert(values,0,0)
        temp.loc[:,v+'_dt']=values
    train_dt.append(temp)

100%|██████████| 3125/3125 [00:43<00:00, 72.49it/s]


In [11]:
test_dt=[]
for i in tqdm(test['id'].unique()):
    temp=test.loc[test['id']==i]
    for v in train.columns[2:]:
        values=jerk_signal(temp[v].values)
        values=np.insert(values,0,0)
        temp.loc[:,v+'_dt']=values
    test_dt.append(temp)

100%|██████████| 782/782 [00:09<00:00, 83.58it/s]


In [12]:
from scipy import fftpack
from numpy.fft import *

def fourier_transform_one_signal(t_signal):
    complex_f_signal= fftpack.fft(t_signal)
    amplitude_f_signal=np.abs(complex_f_signal)
    return amplitude_f_signal

In [13]:
train=pd.concat(train_dt)
train

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,acc_Energy,gy_Energy,gy_acc_Energy,acc_x_dt,acc_y_dt,acc_z_dt,gy_x_dt,gy_y_dt,gy_z_dt,acc_Energy_dt,gy_Energy_dt,gy_acc_Energy_dt
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112,1.146962,12.465436,12.427938,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216,1.200703,12.913284,12.865692,4.080495,-0.980114,-1.699854,44.735403,-429.504677,337.444793,2.687024,22.392358,21.887693
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629,1.217403,13.725729,13.692643,0.845632,0.192961,-3.546937,-196.018888,-249.173073,-4.620631,0.835012,40.622253,41.347563
3,0,3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013,1.209981,15.374021,15.314907,-0.575711,-1.762585,1.908626,316.513181,-473.763910,-121.719195,-0.371100,82.414636,81.113199
4,0,4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234,1.211254,16.074363,16.017964,0.389598,2.130453,-0.365665,78.686055,-215.435892,-25.361098,0.063656,35.017060,35.152822
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1874995,3124,595,-0.712530,-0.658357,0.293707,-29.367857,-104.013664,-76.290437,1.009050,25.963234,25.897316,1.484646,0.303666,0.800069,-150.644663,-34.630282,-8.380088,-0.679712,8.387109,8.432977
1874996,3124,596,-0.683037,-0.658466,0.329223,-30.149089,-101.796809,-76.625087,1.002827,25.784692,25.722482,1.474659,-0.005442,1.775771,-39.061611,110.842743,-16.732496,-0.311171,-8.927089,-8.741727
1874997,3124,597,-0.664730,-0.666625,0.364114,-27.873095,-98.776072,-79.365125,1.006239,25.628060,25.572145,0.915321,-0.407957,1.744566,113.799702,151.036858,-137.001896,0.170620,-7.831611,-7.516832
1874998,3124,598,-0.630534,-0.682565,0.373696,-23.636550,-99.139495,-80.259478,1.001038,25.626266,25.573288,1.709833,-0.796984,0.479107,211.827245,-18.171144,-44.717652,-0.260074,-0.089713,0.057150


In [14]:
fft=[]
for i in tqdm(train['id'].unique()):
    temp=train.loc[train['id']==i]
    for i in train.columns[2:8]:
        temp[i]=fourier_transform_one_signal(temp[i].values)
    fft.append(temp)
train=pd.concat(fft)

100%|██████████| 3125/3125 [00:11<00:00, 273.28it/s]


In [15]:
test=pd.concat(test_dt)
test

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,acc_Energy,gy_Energy,gy_acc_Energy,acc_x_dt,acc_y_dt,acc_z_dt,gy_x_dt,gy_y_dt,gy_z_dt,acc_Energy_dt,gy_Energy_dt,gy_acc_Energy_dt
0,3125,0,-0.628100,-0.160155,0.151487,49.665357,88.435961,13.597668,0.762377,21.878437,21.938882,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,3125,1,-0.462548,0.012462,-0.053726,56.953059,96.185341,16.278458,0.600917,23.367908,23.399763,8.277606,8.630860,-10.260605,364.385137,387.469010,134.039504,-8.072977,74.473561,73.044041
2,3125,2,-0.363481,-0.091789,-0.130004,29.557396,93.836453,13.329043,0.539978,21.440856,21.471510,4.953330,-5.212527,-3.813919,-1369.783175,-117.444408,-147.470749,-3.046963,-96.352572,-96.412634
3,3125,3,-0.351750,-0.239870,-0.193053,23.686172,88.608721,13.449771,0.602338,20.482783,20.533967,0.586560,-7.404041,-3.152460,-293.561181,-261.386607,6.036401,3.118006,-47.903682,-46.877152
4,3125,4,-0.312934,-0.123762,-0.318621,20.410071,85.327707,13.884912,0.598853,19.908061,19.944125,1.940804,5.805371,-6.278395,-163.805080,-164.050703,21.757050,-0.174231,-28.736096,-29.492133
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469195,3906,595,0.104191,-0.784979,0.639513,-10.475346,14.095361,-190.358982,1.011867,33.185050,33.266220,-3.361947,3.580749,0.006738,-771.499131,-849.527466,-8.671145,-2.122263,-9.277974,-9.636798
469196,3906,596,0.103297,-0.758954,0.615687,-25.360272,-8.523018,-180.393291,0.988451,32.158563,32.227841,-0.044724,1.301266,-1.191327,-744.246300,-1130.918984,498.284561,-1.170808,-51.324317,-51.918937
469197,3906,597,0.128294,-0.749389,0.586184,-27.917723,-23.186245,-162.624160,0.973171,30.280197,30.339698,1.249868,0.478247,-1.475149,-127.872573,-733.161359,888.456533,-0.763970,-93.918343,-94.407192
469198,3906,598,0.104130,-0.692731,0.573397,-27.847980,-30.407555,-138.761676,0.935801,27.569491,27.623600,-1.208173,2.832870,-0.639335,3.487160,-361.065463,1193.124196,-1.868499,-135.535264,-135.804895


In [16]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
col=train.columns
scaler = StandardScaler()

train.iloc[:,2:]= scaler.fit_transform(train.iloc[:,2:])
train = pd.DataFrame(data = train,columns =col)

test.iloc[:,2:]= scaler.transform(test.iloc[:,2:])
test = pd.DataFrame(data = test,columns =col)

In [17]:
train

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,acc_Energy,gy_Energy,gy_acc_Energy,acc_x_dt,acc_y_dt,acc_z_dt,gy_x_dt,gy_y_dt,gy_z_dt,acc_Energy_dt,gy_Energy_dt,gy_acc_Energy_dt
0,0,0,27.356382,8.807207,19.465910,0.376992,0.869226,0.150423,0.495681,-0.272719,-0.276391,0.000027,0.000298,-0.000433,0.000347,0.000373,0.000273,0.000101,0.001505,0.001501
1,0,1,-0.054866,0.833464,0.820412,-0.282128,-0.093560,0.011266,0.742974,-0.236152,-0.240632,0.416836,-0.118821,-0.255054,0.032738,-0.349095,0.377085,0.564992,0.166566,0.162871
2,0,2,0.024046,0.315921,0.081086,-0.182551,-0.053585,-0.003708,0.819822,-0.169815,-0.173080,0.086405,0.023750,-0.531727,-0.141582,-0.202368,-0.004887,0.175645,0.300944,0.306341
3,0,3,0.065632,0.117634,-0.040874,-0.194863,0.154242,0.005408,0.785669,-0.035229,-0.040560,-0.058780,-0.213920,0.285459,0.229520,-0.385106,-0.135647,-0.077915,0.609008,0.599518
4,0,4,0.151477,0.300751,0.317742,-0.350724,0.494539,0.154354,0.791528,0.021954,0.016872,0.039823,0.259227,-0.055206,0.057320,-0.174917,-0.028047,0.013483,0.259626,0.260669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1874995,3124,595,0.365037,0.011656,0.845701,0.080839,0.350395,0.112282,-0.138940,0.829394,0.823900,0.151679,0.037205,0.119409,-0.108728,-0.027804,-0.009085,-0.142794,0.063329,0.063674
1874996,3124,596,10.220817,5.476964,7.441373,3.605246,16.530576,11.843241,-0.167578,0.814816,0.809618,0.150658,-0.000363,0.265559,-0.027936,0.090560,-0.018412,-0.065316,-0.064300,-0.062949
1874997,3124,597,0.386337,0.177768,-0.080193,-0.192468,-0.033904,-0.227861,-0.151875,0.802027,0.797338,0.093524,-0.049283,0.260884,0.082744,0.123264,-0.152712,0.035970,-0.056225,-0.053918
1874998,3124,598,0.728823,0.014037,0.350745,0.136284,1.281790,0.403540,-0.175811,0.801880,0.797431,0.174681,-0.096564,0.071332,0.153722,-0.014412,-0.049662,-0.054574,0.000843,0.001922


In [18]:
train.shape

(1875000, 20)

In [19]:
X=np.array(train.iloc[:,2:]).reshape(3125, 600, -1)

In [20]:
X.shape

(3125, 600, 18)

In [21]:
test_x=np.array(test.iloc[:,2:]).reshape(782, 600, -1)
test_x.shape

y = train_labels['label'].values
y = tf.keras.utils.to_categorical(train_labels['label']) 
y.shape

(3125, 61)

##### Standard scaling 적용


##### 모델링

+ CNN, LSTM, CNN+LSTM 등 여러 구조 적용해보다가 CNN에서 Flatten 없이 Global average pooling 한 구조가 가장 성능이 좋아 채택했습니다.


In [23]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_addons as tfa
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM,Bidirectional,Dropout
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from sklearn.model_selection import KFold,StratifiedKFold
from numpy.random import seed
from tensorflow.keras.layers import Activation, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling1D
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

##### 모델 구조 

In [34]:

# model= keras.Sequential()
# model.add(Conv1D(filters=128, kernel_size=9, kernel_initializer='he_uniform' ,padding="same",input_shape=X.shape[1:], activation="relu"))
# model.add(MaxPooling1D())
# model.add(Conv1D(filters=256, kernel_size=9, padding="same",input_shape=X.shape[1:], activation="relu"))
# model.add(MaxPooling1D())
# model.add(Conv1D(filters=128, kernel_size=9, padding="same",input_shape=X.shape[1:], activation="relu"))
# #model.add(MaxPooling1D())
# #model.add(Conv1D(filters=64, kernel_size=9, padding="same",input_shape=X.shape[1:], activation="relu"))
# model.add(GlobalAveragePooling1D())

# #model.add(Flatten())

# model.add(Dense(128, activation="relu"))
# model.add(Dropout(rate=0.1))
# model.add(Dense(64, activation="relu"))
# model.add(Dropout(rate=0.2))


# #model.add(Dense(64, activation="relu"))

# model.add(Dense(61, activation="softmax"))
# model.compile(loss='categorical_crossentropy', optimizer = 'Adam', metrics='accuracy')
# model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_7 (Conv1D)            (None, 600, 128)          20864     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 300, 128)          0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 300, 256)          295168    
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 150, 256)          0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 150, 128)          295040    
_________________________________________________________________
global_average_pooling1d_2 ( (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)              

In [43]:

seed(2021)
tf.random.set_seed(2021)

input_layer = keras.layers.Input(X.shape[1:])
conv1 = keras.layers.Conv1D(filters=128, kernel_size=9, padding='same')(input_layer)
conv1 = keras.layers.BatchNormalization()(conv1)
conv1 = keras.layers.Activation(activation='relu')(conv1)
conv1 = keras.layers.Dropout(rate=0.3)(conv1)

conv2 = keras.layers.Conv1D(filters=256, kernel_size=6, padding='same')(conv1)
conv2 = keras.layers.BatchNormalization()(conv2)
conv2 = keras.layers.Activation('relu')(conv2)
conv2 = keras.layers.Dropout(rate=0.4)(conv2)

conv3 = keras.layers.Conv1D(128, kernel_size=3,padding='same')(conv2)
conv3 = keras.layers.BatchNormalization()(conv3)
conv3 = keras.layers.Activation('relu')(conv3)
conv3 = keras.layers.Dropout(rate=0.5)(conv3)

gap = keras.layers.GlobalAveragePooling1D()(conv3)

output_layer = keras.layers.Dense(61, activation='softmax')(gap)

model = keras.models.Model(inputs=input_layer, outputs=output_layer)

model.compile(loss='categorical_crossentropy', optimizer = 'Adam', metrics='accuracy')
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 600, 18)]         0         
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 600, 128)          20864     
_________________________________________________________________
batch_normalization_3 (Batch (None, 600, 128)          512       
_________________________________________________________________
activation_3 (Activation)    (None, 600, 128)          0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 600, 128)          0         
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 600, 256)          196864    
_________________________________________________________________
batch_normalization_4 (Batch (None, 600, 256)          1024

##### 10-fold StratifiedKFold

In [44]:
skf = StratifiedKFold(n_splits = 10, random_state = 123, shuffle = True)
reLR = ReduceLROnPlateau(patience = 4,verbose = 1,factor = 0.5) 
es =EarlyStopping(monitor='val_loss', patience=10, mode='min')



history = model.fit(X, y, epochs = 100, verbose=1, batch_size=100, validation_split = 0.2, callbacks=[es,reLR])



Epoch 1/100
17/17 [==============================] - 5s 213ms/step - loss: 3.4091 - accuracy: 0.3740 - val_loss: 6.1116 - val_accuracy: 0.0288
Epoch 2/100
17/17 [==============================] - 3s 182ms/step - loss: 2.3918 - accuracy: 0.5168 - val_loss: 4.0041 - val_accuracy: 0.0944
Epoch 3/100
17/17 [==============================] - 3s 176ms/step - loss: 2.0298 - accuracy: 0.5388 - val_loss: 2.5794 - val_accuracy: 0.4800
Epoch 4/100
17/17 [==============================] - 3s 183ms/step - loss: 1.8565 - accuracy: 0.5620 - val_loss: 1.8786 - val_accuracy: 0.5568
Epoch 5/100
17/17 [==============================] - 3s 174ms/step - loss: 1.7284 - accuracy: 0.5800 - val_loss: 1.8508 - val_accuracy: 0.5600
Epoch 6/100
17/17 [==============================] - 3s 182ms/step - loss: 1.6198 - accuracy: 0.6024 - val_loss: 2.0182 - val_accuracy: 0.5648
Epoch 7/100
17/17 [==============================] - 3s 176ms/step - loss: 1.5231 - accuracy: 0.6192 - val_loss: 1.8901 - val_accuracy: 0.5536

In [26]:
from sklearn.model_selection import KFold
from tensorflow.keras import optimizers

In [28]:
# prediction=0
# folds=5
# repeats=50
# epochsize=50
# drop=0.5
# kfold = KFold(n_splits=folds, shuffle=True)

# for k in range(repeats):
#   for train_ind, test_ind in kfold.split(X, y):
#     model = Sequential()
#     model.add(Conv1D (kernel_size=60, filters=256, strides=3, padding='valid',
#                     kernel_initializer='he_uniform', input_shape=X.shape[1:],
#                     activation='relu'))
#     model.add(MaxPooling1D(1, padding = 'valid'))

#     model.add(Conv1D(kernel_size=60, filters=128, activation='relu'))
#     model.add(MaxPooling1D(1, padding = 'valid'))

#     model.add(Conv1D(kernel_size=60, filters=64, activation='relu'))
#     model.add(MaxPooling1D(1, padding = 'valid'))

#     model.add(BatchNormalization())

#     model.add(Dropout(drop))
#     model.add(Flatten())
#     model.add(Dense(1024, activation='relu'))
#     model.add(BatchNormalization())

#     model.add(Dropout(drop))
#     model.add(Dense(1024, activation='relu'))
#     model.add(BatchNormalization())

#     model.add(Dropout(drop))
#     model.add(Dense(1024, activation='relu'))
#     model.add(BatchNormalization())


#     model.add(Dropout(drop))
#     model.add(Dense(61))
#     model.add(Activation('softmax'))

#     adam = optimizers.Adam(lr=0.0001)

#     model.compile(loss='categorical_crossentropy',
#                   optimizer=adam,
#                   metrics=['accuracy'])
    
#     model.fit(np.array(X)[train_ind], y[train_ind], epochs=epochsize, batch_size=64,
#               validation_data=(np.array(X)[test_ind], y[test_ind]))

#     prediction += (model.predict(test_x)/(folds*repeats))

# submission.iloc[:,1:]=prediction
# submission.to_csv('py_answer1.csv', index=False)

ResourceExhaustedError: OOM when allocating tensor with shape[4032,1024] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:RandomUniform]

##### 성능 확인 및 제출

In [45]:
# 8. 모델 평가하기
model.evaluate(X, y)

98/98 [==============================] - 1s 13ms/step - loss: 0.2676 - accuracy: 0.9344


[0.2676210403442383, 0.9344000220298767]

In [46]:
pred = model.predict(test_x)

pred

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [47]:
#submission=pd.read_csv('./sample_submission.csv')
submission.iloc[:,1:]=pred
submission

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,...,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,3125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.732821e-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.287268e-36,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3126,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3127,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.967757e-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3128,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.006263e-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.199374,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3129,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,3902,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
778,3903,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
779,3904,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
780,3905,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.142778e-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
submission.to_csv('sub_chuchu.csv',index=False)